# **Imports**

In [ ]:
%pip install pandas
%pip install nltk
%pip install ipywidgets

import pandas as pd
import os
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import ipywidgets as widgets
from IPython.display import display
from collections import Counter

# **Fazendo a leitura dos dados**

In [ ]:
issues = pd.read_csv("path_to_csv")
comentarios = pd.read_csv("path_to_csv")

# Concatenando o Titulo e a Descrição das issues
issues['Info'] = issues['TituloIssue'] + issues['DescricaoIssue']

# Convertendo todos os tipos para String
issues['Info'] = issues['Info'].astype(str).str.lower()
comentarios['Comentario'] = comentarios['Comentario'].astype(str).str.lower()

# **Tokenizando as issues**

In [ ]:
# Definindo as StopWords
nltk.download('stopwords')
stopWords = set(stopwords.words('english'))


def removerStopWords(palavras):
    return [palavra for palavra in palavras if palavra not in stopWords and palavra.isalpha()]

# Tokenizando
nltk.download('punkt_tab')
from nltk import tokenize

comentariosToK = []
for i in range (comentarios['Comentario'].size):
    comentariosToK.append(removerStopWords(tokenize.word_tokenize(comentarios.loc[i]['Comentario'])))

infoToK = []
for k in range(issues['Info'].size):
    infoToK.append(removerStopWords(tokenize.word_tokenize(issues.loc[k]['Info'])))

# Juntando os vetores
import itertools

comentariosToK = list(itertools.chain(*comentariosToK))
infoToK = list(itertools.chain(*infoToK))


# **Contando os substantivos**

In [ ]:
nltk.download('averaged_perceptron_tagger_eng')
from nltk import pos_tag

tagsComents = pos_tag(comentariosToK)
tagsInfo = pos_tag(infoToK)

subsComents = [palavra for palavra, tag in tagsComents if tag in ['NN', 'NNS', 'NNP', 'NNPS']]
subsInfo = [palavra for palavra, tag in tagsInfo if tag in ['NN', 'NNS', 'NNP', 'NNPS']]

contagemComents = Counter(subsComents)
contagemInfo = Counter(subsInfo)

dfComents = pd.DataFrame(contagemComents.items(), columns=["Substantivo", "Frequência"])
dfInfo = pd.DataFrame(contagemInfo.items(), columns=["Substantivo", "Frequência"])



# **Exibindo os Dataframes**

In [ ]:
dfInfo = dfInfo.sort_values(by='Frequência', ascending=False)
dfInfo

In [ ]:
dfComents = dfComents.sort_values(by='Frequência', ascending=False)
dfComents

# **Exportanto os resultados para um CSV**

In [ ]:
dfComents.to_csv("out/subsComents.csv", index=False)
dfInfo.to_csv("out/subsInfo.csv", index=False)